In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
import numpy as np

In [3]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [4]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


/Users/aa406165/Git/qiskit-core/qiskit/quantum_info/states/statevector.py:444: DeprecationWarning: The SparsePauliOp.table method is deprecated as of Qiskit Terra 0.19.0 and will be removed no sooner than 3 months after the releasedate. Use SparsePauliOp.paulis method instead.
  for z, x, coeff in zip(oper.table.Z, oper.table.X, oper.coeffs)


## ExpectationValue class

In [5]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [6]:
from qiskit.providers.aer import AerSimulator

backend = AerSimulator()

In [7]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5], shos=1000)

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=1.908203125, variance=11.164134979248047, confidence_interval=(1.7482299887973152, 2.0681762612026846))

In [8]:
# pre-binding

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=1.896484375, variance=11.11996078491211, confidence_interval=(1.737127799850259, 2.055840950149741))

In [9]:
# multi prameters

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]], shots=3000)

no max_experiments for this backend: aer_simulator


ExpectationValueArrayResult(values=array([1.83466667, 0.07      ]), variances=array([11.19212356, 10.30434444]), confidence_intervals=array([[ 1.74132206,  1.92801127],
       [-0.02332028,  0.16332028]]))

In [10]:
# can pass ndarray
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate(np.array([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]]), shots=8192)

no max_experiments for this backend: aer_simulator


ExpectationValueArrayResult(values=array([1.86230469, 0.12402344]), variances=array([11.12684155, 10.26732111]), confidence_intervals=array([[1.8061174 , 1.91849198],
       [0.06784577, 0.1802011 ]]))

### Exact simulation by SaveExpectationValueVariance

In [11]:
from qiskit.evaluators import ExactExpectationValue

In [12]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.8420921273153708, variance=6.432763522806668, confidence_interval=None)

In [13]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
expval.evaluate(np.array([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]]))

ExpectationValueArrayResult(values=array([1.84209213, 0.15611897]), variances=array([6.43276352, 5.35681431]), confidence_intervals=array([None, None], dtype=object))

### Transpiled Circuits

In [14]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.set_transpile_options(initial_layout=[3, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
    print()
print(expval.evaluate([0, 1, 1, 2, 3, 5]))

global phase: π
                                                                »
ancilla_0 -> 0 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 1 ─────────────────────────────────────────────────»
               ┌────┐┌──────────────┐┌────┐┌────────┐┌───┐┌────┐»
      q_1 -> 2 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤└─┬─┘├────┤»
      q_0 -> 3 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘     └────┘»
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
         c4: 2/═════════════════════════════════════════════════»
                                                                »
«                                                                          »
«ancilla_0 -> 0 ─────────────────────────────────

### Transpile options and Run options

In [15]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=1_000_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.1926452310888873, variance=12.759702334055865, confidence_interval=(1.1869819463723348, 1.1983085158054398))

In [16]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.19384765625, variance=12.713602781295776, confidence_interval=(1.131434915015488, 1.256260397484512))

In [17]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=0.8933333333333333, variance=13.013244444444446, confidence_interval=(0.5623285489914652, 1.2243381176752015))

### Composite Evaluator

In [18]:
from qiskit.evaluators import JointEvaluator

In [19]:
len(expval.transpiled_circuits)

3

In [20]:
joint_evaluator = JointEvaluator([expval, expval, expval])  # can be different evaluator

In [21]:
len(joint_evaluator.transpiled_circuits)  # 3 × 3

9

In [22]:
joint_evaluator.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13]])

CompositeResult(items=[ExpectationValueResult(value=1.1357421875, variance=12.831414580345154, confidence_interval=(1.0729446840872225, 1.1985396909127775)), ExpectationValueResult(value=1.1357421875, variance=12.831414580345154, confidence_interval=(1.0729446840872225, 1.1985396909127775)), ExpectationValueResult(value=1.1357421875, variance=12.831414580345154, confidence_interval=(1.0729446840872225, 1.1985396909127775))])

### Mitigator

In [23]:
from qiskit.evaluators.backends import ReadoutErrorMitigation

backend = AerSimulator.from_backend(FakeBogota())
mit = ReadoutErrorMitigation(
    backend, mitigation="tensored", refresh=1, shots=2000, mit_pattern=[[0], [1]]
)
expval_raw = PauliExpectationValue(ansatz, observable, backend=backend)
expval_mit = PauliExpectationValue(ansatz, observable, backend=mit)
shots = 2000
print(
    f"w/o mitigation shots={shots}, result={expval_raw.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)
print(
    f"w/  mitigation shots={shots}, result={expval_mit.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}"
)

w/o mitigation shots=2000, result=ExpectationValueResult(value=1.096, variance=12.84849, confidence_interval=(0.9688127758048671, 1.223187224195133))
w/  mitigation shots=2000, result=ExpectationValueResult(value=1.4011162928411947, variance=12.322563554620057, confidence_interval=(1.2774786683159185, 1.5247539173664708))


In [26]:
exact_expval = ExactExpectationValue(ansatz, observable, backend=AerSimulator())
exact_findiff = FiniteDiffGradient(exact_expval, 1e-8)
print(f"fin diff of exact {exact_findiff.evaluate([0, 1, 1, 2, 3, 5]).values}")

NameError: name 'FiniteDiffGradient' is not defined

NameError: name 'QubitConverter' is not defined